In [ ]:
!nvidia-smi

Wed Dec 18 18:45:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install pyngrok

In [ ]:
# # First, install ngrok and other requirements
!pip install flask googletrans==3.1.0a0 gtts peft transformers torch accelerate
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin

# Ensure you have your ngrok authtoken (you'll need to sign up at ngrok.com to get one)
import getpass
print("Enter your ngrok authtoken (sign up at ngrok.com):")
authtoken = getpass.getpass()
!ngrok authtoken $authtoken

import os
import random
from flask import Flask, render_template, request, jsonify
from googletrans import Translator
from gtts import gTTS
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, GenerationConfig
import torch
from threading import Thread
from flask import Response
import json
import requests
import time

# Initialize Flask app
app = Flask(__name__)

# Initialize the Falcon model
class BookFinetunedFalcon:
    def __init__(self, model_path, device="cuda"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoPeftModelForCausalLM.from_pretrained(
            model_path,
            low_cpu_mem_usage=True,
            return_dict=True,
            torch_dtype=torch.float16,
            device_map=device
        )

        self.model.eval()

        self.generation_config = GenerationConfig(
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            max_new_tokens=512,
            min_new_tokens=50,
            num_beams=1,
            repetition_penalty=1.2,
            length_penalty=1.0,
            no_repeat_ngram_size=3,
            pad_token_id=self.tokenizer.eos_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
            early_stopping=True
        )

    def generate_response(self, prompt, max_length=None):
        formatted_prompt = (
            "### Human: Given the following question about pregnancy and maternity, "
            "provide a detailed and accurate response:\n\n"
            f"{prompt}\n\n"
            "### Assistant: Based on medical knowledge about pregnancy and maternity, "
            "here is my response:"
        )

        inputs = self.tokenizer(
            formatted_prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=1024
        ).to(self.model.device)

        if max_length:
            self.generation_config.max_new_tokens = max_length

        try:
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    generation_config=self.generation_config
                )

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response.split("### Assistant:")[-1].strip()
            response = response.replace("###", "").strip()
            return response

        except Exception as e:
            return f"Error generating response: {str(e)}"

# Initialize global variables
model = None
translator = Translator()

# Language mapping
LANGUAGES = {
    'hi': {'name': 'Hindi', 'code': 'hi'},
    'ta': {'name': 'Tamil', 'code': 'ta'},
    'te': {'name': 'Telugu', 'code': 'te'},
    'ml': {'name': 'Malayalam', 'code': 'ml'},
    'kn': {'name': 'Kannada', 'code': 'kn'},
    'mr': {'name': 'Marathi', 'code': 'mr'},
    'gu': {'name': 'Gujarati', 'code': 'gu'},
    'bn': {'name': 'Bengali', 'code': 'bn'}
}

def safe_translate(text, src_lang, dest_lang):
    try:
        # Get response from Falcon model (in English)
        global model
        if model is None:
            model = BookFinetunedFalcon(
                "onkar234567/falcon-maternity-model",
                device="cuda" if torch.cuda.is_available() else "cpu"
            )

        # First translate input to English if not already in English
        if src_lang != 'en':
            to_english = translator.translate(text, src=src_lang, dest='en')
            english_text = to_english.text
        else:
            english_text = text

        # Get model response
        model_response = model.generate_response(english_text)

        # Translate model response to target language
        if dest_lang != 'en':
            translated_response = translator.translate(model_response, src='en', dest=dest_lang)
            final_response = translated_response.text
        else:
            final_response = model_response

        return {
            'original_input': text,
            'english_translation': english_text,
            'translated_response': final_response
        }
    except Exception as e:
        return {
            'original_input': text,
            'english_translation': f"Translation error: {str(e)}",
            'translated_response': f"Error: {str(e)}"
        }

# Create HTML template string
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>MaternAI</title>
    <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet">
    <style>
        .chat-container {
            max-height: 500px;
            overflow-y:auto;
        }
        .message {
            max-width: 80%;
        }
        .user-message {
            background-color: #E6F2FF;
            align-self: flex-end;
        }
        .bot-message {
            background-color: #F0F0F0;
            align-self: flex-start;
        }
    </style>
</head>
<body class="bg-gray-100">
    <div class="container mx-auto px-4 py-8 max-w-2xl">
        <div class="bg-white shadow-lg rounded-lg overflow-hidden">
            <div class="p-4 bg-blue-500 text-white text-center">
                <h1 class="text-2xl font-bold">MaternAI</h1>
                <p class="text-sm mt-2">Ask questions about pregnancy and maternity care in your preferred language</p>
            </div>

            <div class="p-4 flex space-x-2">
                <select id="language" class="flex-grow p-2 border rounded">
                    <option value="hi-IN">Hindi (India)</option>
                    <option value="ta-IN">Tamil (India)</option>
                    <option value="te-IN">Telugu (India)</option>
                    <option value="ml-IN">Malayalam (India)</option>
                    <option value="kn-IN">Kannada (India)</option>
                    <option value="mr-IN">Marathi (India)</option>
                    <option value="gu-IN">Gujarati (India)</option>
                    <option value="bn-IN">Bengali (India)</option>
                </select>
                <button id="startRecording" class="bg-green-500 text-white px-4 py-2 rounded hover:bg-green-600">
                    🎤 Start Voice
                </button>
                <button id="stopRecording" disabled class="bg-red-500 text-white px-4 py-2 rounded hover:bg-red-600">
                    ◼ Stop
                </button>
            </div>

            <div id="status" class="p-2 text-center text-gray-600"></div>

            <div id="chatContainer" class="chat-container p-4 space-y-4 flex flex-col">
                <div class="message bot-message self-start p-3 rounded-lg">
                    👋 Hi! I'm your multilingual maternity assistant. Ask me any questions about pregnancy and maternal health.
                </div>
            </div>

            <div class="p-4 border-t">
                <div class="flex space-x-2">
                    <input type="text" id="textInput" placeholder="Type your question about pregnancy or maternity..."
                           class="flex-grow p-2 border rounded">
                    <button id="sendText" class="bg-blue-500 text-white px-4 py-2 rounded hover:bg-blue-600">
                        Send
                    </button>
                </div>
            </div>
        </div>

        <audio id="responseAudio" class="hidden"></audio>
    </div>
        <script>
        const startButton = document.getElementById('startRecording');
        const stopButton = document.getElementById('stopRecording');
        const statusDiv = document.getElementById('status');
        const chatContainer = document.getElementById('chatContainer');
        const languageSelect = document.getElementById('language');
        const responseAudio = document.getElementById('responseAudio');
        const textInput = document.getElementById('textInput');
        const sendTextButton = document.getElementById('sendText');

        // Map language codes to translation codes
        const LANGUAGE_MAP = {
            'hi-IN': 'hi',
            'ta-IN': 'ta',
            'te-IN': 'te',
            'ml-IN': 'ml',
            'kn-IN': 'kn',
            'mr-IN': 'mr',
            'gu-IN': 'gu',
            'bn-IN': 'bn'
        };

        // Check browser support
        const SpeechRecognition = window.SpeechRecognition || window.webkitSpeechRecognition;
        if (!SpeechRecognition) {
            statusDiv.textContent = 'Speech recognition not supported';
            startButton.disabled = true;
        }

        const recognition = new SpeechRecognition();
        recognition.continuous = false;
        recognition.interimResults = false;

        function addMessageToChatContainer(message, type, audioPaths = null) {
            const messageElement = document.createElement('div');
            messageElement.classList.add(
                'message',
                type === 'user' ? 'user-message' : 'bot-message',
                'self-' + (type === 'user' ? 'end' : 'start'),
                'p-3', 'rounded-lg'
            );
            messageElement.textContent = message;

            // Add audio playback button if audio is available
            if (audioPaths && audioPaths.audio_path) {
                const playButton = document.createElement('button');
                playButton.textContent = '🔊 Listen';
                playButton.classList.add('ml-2', 'bg-blue-500', 'text-white', 'px-2', 'py-1', 'rounded', 'text-sm');
                playButton.addEventListener('click', () => {
                    responseAudio.src = audioPaths.audio_path;
                    responseAudio.play();
                });
                messageElement.appendChild(playButton);
            }

            chatContainer.appendChild(messageElement);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }

        function processInput(voiceInput) {
            const languageCode = languageSelect.value;
            const translationCode = LANGUAGE_MAP[languageCode];

            statusDiv.textContent = 'Processing input...';

            addMessageToChatContainer(voiceInput, 'user');

            const formData = new FormData();
            formData.append('voice_input', voiceInput);
            formData.append('language', translationCode);

            fetch('/process', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                addMessageToChatContainer(data.translated_response, 'bot', data);

                statusDiv.textContent = 'Processing complete';
            })
            .catch(error => {
                addMessageToChatContainer('Sorry, an error occurred.', 'bot');
                statusDiv.textContent = 'Error processing input';
                console.error('Error:', error);
            });
        }

        // Set language for speech recognition based on dropdown
        recognition.onstart = () => {
            const selectedLang = languageSelect.value;
            recognition.lang = selectedLang;

            statusDiv.textContent = 'Listening... Speak now';
            startButton.disabled = true;
            stopButton.disabled = false;
        };

        recognition.onresult = (event) => {
            const voiceInput = event.results[0][0].transcript;
            processInput(voiceInput);
        };

        recognition.onerror = (event) => {
            statusDiv.textContent = 'Error occurred in recognition: ' + event.error;
            startButton.disabled = false;
            stopButton.disabled = true;
        };

        recognition.onend = () => {
            statusDiv.textContent = 'Voice input ended';
            startButton.disabled = false;
            stopButton.disabled = true;
        };

        startButton.addEventListener('click', () => {
            try {
                recognition.start();
            } catch (error) {
                statusDiv.textContent = 'Error starting recognition: ' + error;
            }
        });

        stopButton.addEventListener('click', () => {
            recognition.stop();
        });

        sendTextButton.addEventListener('click', () => {
            const inputText = textInput.value.trim();
            if (inputText) {
                processInput(inputText);
                textInput.value = '';
            }
        });

        textInput.addEventListener('keypress', (e) => {
            if (e.key === 'Enter') {
                const inputText = textInput.value.trim();
                if (inputText) {
                    processInput(inputText);
                    textInput.value = '';
                }
            }
        });
    </script>
</body>
</html>
"""

@app.route('/')
def index():
    return html_template

@app.route('/process', methods=['POST'])
def process_input():
    try:
        voice_input = request.form.get('voice_input', '').strip()
        target_language = request.form.get('language', 'hi')

        translation_result = safe_translate(voice_input, target_language, target_language)

        tts = gTTS(text=translation_result['translated_response'], lang=target_language)

        os.makedirs('static', exist_ok=True)
        audio_path = f'static/response_{target_language}_{random.randint(1000,9999)}.mp3'
        tts.save(audio_path)

        translation_result['audio_path'] = audio_path

        return jsonify(translation_result)

    except Exception as e:
        return jsonify({'error': str(e)}), 500

def run_ngrok():
    from pyngrok import ngrok

    # Set up ngrok tunnel
    public_url = ngrok.connect(5000)
    print(f' * Public URL: {public_url}')

def run_app():
    app.run(port=5000)

# Main execution
if __name__ == '__main__':
    # Initialize model
    print("Initializing Falcon model...")
    model = BookFinetunedFalcon(
        "onkar234567/falcon-maternity-model",
        device="cuda" if torch.cuda.is_available() else "cpu"
    )

    # Start ngrok in a separate thread
    from pyngrok import ngrok
    ngrok_thread = Thread(target=run_ngrok)
    ngrok_thread.start()

    # Start Flask app
    print("Starting Flask application...")
    app.run(port=5000)

--2024-12-18 19:00:29--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 35.71.179.82, 75.2.60.68, 99.83.220.108, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.71.179.82|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14796857 (14M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.1’

ngrok-v3-stable-lin 100%[===================>]  14.11M  71.5MB/s    in 0.2s    

2024-12-18 19:00:30 (71.5 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.1’ saved [14796857/14796857]

ngrok
Enter your ngrok authtoken (sign up at ngrok.com):
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Initializing Falcon model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Starting Flask application...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Public URL: NgrokTunnel: "https://e156-34-148-67-68.ngrok-free.app" -> "http://localhost:5000"


INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 19:02:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 19:02:30] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 19:03:20] "POST /process HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 19:03:28] "GET /static/response_hi_5441.mp3 HTTP/1.1" 206 -
INFO:werkzeug:127.0.0.1 - - [18/Dec/2024 19:03:29] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset 